In [37]:
import pandas as ps
import requests
import json
import re
import time
import pip
import subprocess

In [38]:
%%bash 
find /home/afollette/Finder_Data_Repositories/pdxfinder-data-release-head/data/UPDOG -name "*mut*tsv" -exec bash -c "grep "\'"rs[0-9]\+"\'" {} | shuf --head-count=5 -"  \; > ./rsidSample 


In [39]:
sampleDf = ps.read_csv("rsidSample", sep='\t', names=["model_id","sample_id","sample_origin","host_strain_nomenclature","passage","symbol","biotype","coding_sequence_change","variant_class","codon_change","amino_acid_change","consequence","functional_prediction","read_depth","allele_frequency","chromosome","seq_start_position","ref_allele","alt_allele","ucsc_gene_id","ncbi_gene_id","ncbi_transcript_id","ensembl_gene_id","ensembl_transcript_id","variation_id","genome_assembly","platform"])
sampleDf['variation_id'].str.extract('(rs[0-9]{0,9})').to_csv('cleaned_rsids', sep='\t', index=False)

In [80]:
%%bash
/home/afollette/jupyter_notebooks/integrationTest/spdi_batch.py -i cleaned_rsids -t RS > queryResults
print('download cross-reference complete')

bash: line 2: syntax error near unexpected token `'download cross-reference complete''
bash: line 2: `print('download cross-reference complete')'


CalledProcessError: Command 'b"/home/afollette/jupyter_notebooks/integrationTest/spdi_batch.py -i cleaned_rsids -t RS > queryResults\nprint('download cross-reference complete')\n"' returned non-zero exit status 2.

In [15]:
apiResults = ps.read_csv('queryResults', header=False)

TypeError: Passing a bool to header is invalid. Use header=None for no header or header=int or list-like of ints to specify the row(s) making up the column names

In [118]:
def checkRsid(row, modifier):
    modelId = row.loc['model_id']
    sampleId = row.loc['sample_id']
    variations = row.loc['variation_id']
    rsid = re.match("rs[0-9]+", variations).group(0)
    chro = row.loc['chromosome']
    pos = row.loc['seq_start_position']
    ref = row.loc['ref_allele']
    alt = row.loc['alt_allele']
    uri = "https://run.opencravat.org/submit/annotate?chrom={0}&pos={1}&ref_base={2}&alt_base={3}&annotators=dbsnp".format(chro,pos+modifier,ref,alt)
    response = requests.get(uri)
    if(response.status_code is not 200): 
        print(response.status_code)
        time.sleep(backOff)
        backoff = backoff+10
    else:
        dbsnp = response.json()['dbsnp']
        checkedRs = None
        if dbsnp:
            checkedRs = dbsnp['snp']
        doRsMatch = rsid == checkedRs
        resultTuple = modifier,rsid,checkedRs,doRsMatch
        print(resultTuple)
        resultDict[modelId + "_" + sampleId] = resultTuple


In [119]:
def toggleCoordinates(row):
    checkRsid(row, 1)
    checkRsid(row, 0)
    checkRsid(row, -1)

In [120]:
resultDict = {}
backoff=10
sampleDf.apply(lambda x: toggleCoordinates(x), axis=1)


(1, 'rs3210731', None, False)
(0, 'rs3210731', None, False)
(-1, 'rs3210731', None, False)
(1, 'rs35283192', None, False)
(0, 'rs35283192', None, False)
(-1, 'rs35283192', None, False)
(1, 'rs772765696', None, False)
(0, 'rs772765696', None, False)
(-1, 'rs772765696', None, False)
(1, 'rs28763975', None, False)
(0, 'rs28763975', None, False)
(-1, 'rs28763975', None, False)
(1, 'rs3832131', None, False)
(0, 'rs3832131', None, False)
(-1, 'rs3832131', None, False)
(1, 'rs3784632', None, False)
(0, 'rs3784632', None, False)
(-1, 'rs3784632', None, False)
(1, 'rs150300324', None, False)
(0, 'rs150300324', None, False)
(-1, 'rs150300324', None, False)
(1, 'rs34842500', None, False)
(0, 'rs34842500', None, False)
(-1, 'rs34842500', None, False)
(1, 'rs2270425', None, False)
(0, 'rs2270425', None, False)
(-1, 'rs2270425', None, False)
(1, 'rs2274738', None, False)
(0, 'rs2274738', None, False)
(-1, 'rs2274738', None, False)
(1, 'rs150617', None, False)
(0, 'rs150617', None, False)
(-1, 'rs150

(1, 'rs148216598', None, False)
(0, 'rs148216598', None, False)
(-1, 'rs148216598', None, False)
(1, 'rs1265188394', None, False)
(0, 'rs1265188394', None, False)
(-1, 'rs1265188394', None, False)
(1, 'rs148726891', None, False)
(0, 'rs148726891', None, False)
(-1, 'rs148726891', None, False)
(1, 'rs758917875', None, False)
(0, 'rs758917875', None, False)
(-1, 'rs758917875', None, False)
(1, 'rs766185415', None, False)
(0, 'rs766185415', None, False)
(-1, 'rs766185415', None, False)
(1, 'rs143092669', None, False)
(0, 'rs143092669', None, False)
(-1, 'rs143092669', None, False)
(1, 'rs796096347', None, False)
(0, 'rs796096347', None, False)
(-1, 'rs796096347', None, False)
(1, 'rs201672509', None, False)
(0, 'rs201672509', None, False)
(-1, 'rs201672509', None, False)
(1, 'rs202060088', None, False)
(0, 'rs202060088', None, False)
(-1, 'rs202060088', None, False)
(1, 'rs116768871', None, False)
(0, 'rs116768871', None, False)
(-1, 'rs116768871', None, False)
(1, 'rs201336331', None, Fa

(0, 'rs775400514', None, False)
(-1, 'rs775400514', None, False)
(1, 'rs146200489', None, False)
(0, 'rs146200489', None, False)
(-1, 'rs146200489', None, False)
(1, 'rs76437836', None, False)
(0, 'rs76437836', None, False)
(-1, 'rs76437836', None, False)
(1, 'rs1200876', None, False)
(0, 'rs1200876', None, False)
(-1, 'rs1200876', None, False)
(1, 'rs200909650', None, False)
(0, 'rs200909650', None, False)
(-1, 'rs200909650', None, False)
(1, 'rs61900347', None, False)
(0, 'rs61900347', None, False)
(-1, 'rs61900347', None, False)
(1, 'rs71891516', None, False)
(0, 'rs71891516', None, False)
(-1, 'rs71891516', None, False)
(1, 'rs4024453', None, False)
(0, 'rs4024453', None, False)
(-1, 'rs4024453', None, False)
(1, 'rs191563773', None, False)
(0, 'rs191563773', None, False)
(-1, 'rs191563773', None, False)
(1, 'rs762326399', None, False)
(0, 'rs762326399', None, False)
(-1, 'rs762326399', None, False)
(1, 'rs2883478', None, False)
(0, 'rs2883478', None, False)
(-1, 'rs2883478', None,

(0, 'rs2073447', None, False)
(-1, 'rs2073447', None, False)
(1, 'rs8056693', None, False)
(0, 'rs8056693', None, False)
(-1, 'rs8056693', None, False)
(1, 'rs1302915950', None, False)
(0, 'rs1302915950', None, False)
(-1, 'rs1302915950', None, False)
(1, 'rs150485202', None, False)
(0, 'rs150485202', None, False)
(-1, 'rs150485202', None, False)
(1, 'rs745551659', None, False)
(0, 'rs745551659', None, False)
(-1, 'rs745551659', None, False)
(1, 'rs773521289', None, False)
(0, 'rs773521289', None, False)
(-1, 'rs773521289', None, False)
(1, 'rs192345324', None, False)
(0, 'rs192345324', None, False)
(-1, 'rs192345324', None, False)
(1, 'rs9328936', None, False)
(0, 'rs9328936', None, False)
(-1, 'rs9328936', None, False)
(1, 'rs202216852', None, False)
(0, 'rs202216852', None, False)
(-1, 'rs202216852', None, False)
(1, 'rs138411966', None, False)
(0, 'rs138411966', None, False)
(-1, 'rs138411966', None, False)
(1, 'rs78032017', None, False)
(0, 'rs78032017', None, False)
(-1, 'rs78032

(0, 'rs746206549', None, False)
(-1, 'rs746206549', None, False)
(1, 'rs748845914', None, False)
(0, 'rs748845914', None, False)
(-1, 'rs748845914', None, False)
(1, 'rs61750923', None, False)
(0, 'rs61750923', None, False)
(-1, 'rs61750923', None, False)
(1, 'rs118124881', None, False)
(0, 'rs118124881', None, False)
(-1, 'rs118124881', None, False)
(1, 'rs72822197', None, False)
(0, 'rs72822197', None, False)
(-1, 'rs72822197', None, False)
(1, 'rs73163757', None, False)
(0, 'rs73163757', None, False)
(-1, 'rs73163757', None, False)
(1, 'rs771320840', None, False)
(0, 'rs771320840', None, False)
(-1, 'rs771320840', None, False)
(1, 'rs74408075', None, False)
(0, 'rs74408075', None, False)
(-1, 'rs74408075', None, False)
(1, 'rs189033097', None, False)
(0, 'rs189033097', None, False)
(-1, 'rs189033097', None, False)
(1, 'rs777700791', None, False)
(0, 'rs777700791', None, False)
(-1, 'rs777700791', None, False)
(1, 'rs143914870', None, False)
(0, 'rs143914870', None, False)
(-1, 'rs14

...

(1, 'rs10171441', None, False)
(0, 'rs10171441', None, False)
(-1, 'rs10171441', None, False)
(1, 'rs1220206200', None, False)
(0, 'rs1220206200', None, False)
(-1, 'rs1220206200', None, False)
(1, 'rs142288648', None, False)
(0, 'rs142288648', None, False)
(-1, 'rs142288648', None, False)
(1, 'rs1127332', None, False)
(0, 'rs1127332', None, False)
(-1, 'rs1127332', None, False)
(1, 'rs145426834', None, False)
(0, 'rs145426834', None, False)
(-1, 'rs145426834', None, False)
(1, 'rs80359852', None, False)
(0, 'rs80359852', None, False)
(-1, 'rs80359852', None, False)
(1, 'rs4126496', None, False)
(0, 'rs4126496', None, False)
(-1, 'rs4126496', None, False)
(1, 'rs201840029', None, False)
(0, 'rs201840029', None, False)
(-1, 'rs201840029', None, False)
(1, 'rs1388920658', None, False)
(0, 'rs1388920658', None, False)
(-1, 'rs1388920658', None, False)
(1, 'rs769531256', None, False)
(0, 'rs769531256', None, False)
(-1, 'rs769531256', None, False)
(1, 'rs1553270812', None, False)
(0, 'rs15

(1, 'rs62548542', None, False)
(0, 'rs62548542', None, False)
(-1, 'rs62548542', None, False)
(1, 'rs3809727', None, False)
(0, 'rs3809727', None, False)
(-1, 'rs3809727', None, False)
(1, 'rs149766175', None, False)
(0, 'rs149766175', None, False)
(-1, 'rs149766175', None, False)
(1, 'rs920316390', None, False)
(0, 'rs920316390', None, False)
(-1, 'rs920316390', None, False)
(1, 'rs200063908', None, False)
(0, 'rs200063908', None, False)
(-1, 'rs200063908', None, False)
(1, 'rs201967730', None, False)
(0, 'rs201967730', None, False)
(-1, 'rs201967730', None, False)
(1, 'rs200441040', None, False)
(0, 'rs200441040', None, False)
(-1, 'rs200441040', None, False)
(1, 'rs770420386', None, False)
(0, 'rs770420386', None, False)
(-1, 'rs770420386', None, False)
(1, 'rs758305500', None, False)
(0, 'rs758305500', None, False)
(-1, 'rs758305500', None, False)
(1, 'rs1162048820', None, False)
(0, 'rs1162048820', None, False)
(-1, 'rs1162048820', None, False)
(1, 'rs15984', None, False)
(0, 'rs1

(-1, 'rs11007612', None, False)
(1, 'rs3960693', None, False)
(0, 'rs3960693', None, False)
(-1, 'rs3960693', None, False)
(1, 'rs76907133', None, False)
(0, 'rs76907133', None, False)
(-1, 'rs76907133', None, False)
(1, 'rs375120330', None, False)
(0, 'rs375120330', None, False)
(-1, 'rs375120330', None, False)
(1, 'rs1471223767', None, False)
(0, 'rs1471223767', None, False)
(-1, 'rs1471223767', None, False)
(1, 'rs1479690146', None, False)
(0, 'rs1479690146', None, False)
(-1, 'rs1479690146', None, False)
(1, 'rs776090319', None, False)
(0, 'rs776090319', None, False)
(-1, 'rs776090319', None, False)
(1, 'rs3809727', None, False)
(0, 'rs3809727', None, False)
(-1, 'rs3809727', None, False)
(1, 'rs71231749', None, False)
(0, 'rs71231749', None, False)
(-1, 'rs71231749', None, False)
(1, 'rs56222179', None, False)
(0, 'rs56222179', None, False)
(-1, 'rs56222179', None, False)
(1, 'rs201173374', None, False)
(0, 'rs201173374', None, False)
(-1, 'rs201173374', None, False)
(1, 'rs155388

(1, 'rs766890316', None, False)
(0, 'rs766890316', None, False)
(-1, 'rs766890316', None, False)
(1, 'rs6666178', None, False)
(0, 'rs6666178', None, False)
(-1, 'rs6666178', None, False)
(1, 'rs3830472', None, False)
(0, 'rs3830472', None, False)
(-1, 'rs3830472', None, False)
(1, 'rs199966834', None, False)
(0, 'rs199966834', None, False)
(-1, 'rs199966834', None, False)
(1, 'rs61758784', None, False)
(0, 'rs61758784', None, False)
(-1, 'rs61758784', None, False)
(1, 'rs561172015', None, False)
(0, 'rs561172015', None, False)
(-1, 'rs561172015', None, False)
(1, 'rs150224836', None, False)
(0, 'rs150224836', None, False)
(-1, 'rs150224836', None, False)
(1, 'rs74994984', None, False)
(0, 'rs74994984', None, False)
(-1, 'rs74994984', None, False)
(1, 'rs148277228', None, False)
(0, 'rs148277228', None, False)
(-1, 'rs148277228', None, False)
(1, 'rs1628172', None, False)
(0, 'rs1628172', None, False)
(-1, 'rs1628172', None, False)
(1, 'rs760923576', None, False)
(0, 'rs760923576', Non

NameError: ("name 'backOff' is not defined", 'occurred at index 2527')

In [ ]:
import csv

In [125]:
sampleDf.iloc[:,9:]

,codon_change,amino_acid_change,consequence,functional_prediction,read_depth,allele_frequency,chromosome,seq_start_position,ref_allele,alt_allele,ucsc_gene_id,ncbi_gene_id,ncbi_transcript_id,ensembl_gene_id,ensembl_transcript_id,variation_id,genome_assembly,platform
0,Atg/Ttg,M1424L,missense_variant,PolyPhen: benign(0) | SIFT: tolerated(0.38),98.0,0.500000,1,232428551,T,A,NaN,57568.0,XM_017001896.1,ENSG00000116991,ENST00000366630,rs3210731,Hg38,Illumina HiSeq (50bp PE)
1,-,NaN,intron_variant,NaN,204.0,0.147059,9,128170117,AAC,A,NaN,25792.0,NM_001131015.2,ENSG00000148337,ENST00000538431,"rs35283192,rs59232258",Hg38,Illumina HiSeq (50bp PE)
2,Caa/Taa,Q81*,stop_gained,NaN,124.0,0.129032,9,128509017,C,T,NaN,2733.0,NM_001003722.1,ENSG00000119392,ENST00000309971,rs772765696,Hg38,Illumina HiSeq (50bp PE)
3,gaC/gaG,D506E,missense_variant,PolyPhen: benign(0.003) | SIFT: tolerated(0.55),21.0,0.809524,6,111661835,G,C,NaN,2534.0,NM_002037.5,ENSG00000010810,ENST00000354650,rs28763975,Hg38,Illumina HiSeq (50bp PE)
4,-,NaN,intron_variant,NaN,37.0,0.432432,2,206587427,GT,G,NaN,8745.0,NM_003812.4,ENSG00000114948,ENST00000264377,"rs3832131,TMP_ESP_2_207452152_207452152",Hg38,Illumina HiSeq (50bp PE)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3808,cCg/cTg,P1185L,missense_variant,PolyPhen: benign(0.014) | SIFT: deleterious(0.05),NaN,0.120000,chr9,135791848,C,T,NaN,57582.0,NM_020822.3,ENSG00000107147,ENST00000371757,rs544495873,Hg38,Macrogen WES
3809,Gac/Aac,D3324N,missense_variant,PolyPhen: probably_damaging(1) | SIFT: deleter...,370.0,0.421622,4,125450986,G,A,NaN,79633.0,XM_011532236.2,ENSG00000196159,ENST00000394329,"rs1304140681,COSM5463959,COSM5463960",GRCh38,243-GEA gene panel
3810,Cag/Tag,Q305*,stop_gained,NaN,989.0,997.000000,19,1221999,C,T,NaN,6794.0,NM_000455.4,ENSG00000118046,ENST00000326873,"rs1131690945,COSM6936070,COSM6936071",GRCh38,Targeted NGS
3811,Aac/Gac,N564D,missense_variant,PolyPhen: possibly_damaging(0.671) | SIFT: del...,2175.0,654.000000,5,68295269,A,G,NaN,5295.0,XM_005248542.3,ENSG00000145675,ENST00000521381,"rs1057519841,COSM1220622,COSM1220623,COSM42912",GRCh38,Targeted NGS


In [ ]:
writer = csv.DictWriter(open("randomRsTest", 'w+'))